In [1]:
#Токен ВК
token = 'e183c92ce183c92ce183c92c55e1f9f915ee183e183c92c8018381b1da2c2188c944592'
#Информационный ресурс, посвященный фондовому рынку
domain_main = 'investingcom'
#Выбрали криптовалюту как содержательную тематику
domain_special = 'investorlife1'

In [93]:
import pandas as pd
import yfinance as yf
from datetime import datetime
import numpy as np
import matplotlib.pyplot as pl
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor

In [2]:
# Импорты
from requests import get
from tqdm.notebook import tqdm
import re
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from collections import Counter
import pandas as pd
import yfinance as yf
from matplotlib import pyplot as plt

#Забираем информацию по api
params = {'access_token': token,
          'v':'5.130',
          'domain':domain_main,
          'count':1
         }
inf_count = get('https://api.vk.com/method/wall.get', params = params).json()['response']['count']
offset_list = list(range(0,inf_count+1,100))
params['count'] = 100
text_list = []
date_list = []
for off in tqdm(offset_list):
    params['offset'] = off
    inf = get('https://api.vk.com/method/wall.get', params = params).json()['response']['items']
    for post in inf:
        if 'text' in list(post.keys()):
            text_list.append(post['text'])
        if 'date' in list(post.keys()):
            date_list.append(post['date'])
# У нас получается 2 списка с данными время и текстом             
stop_words = stopwords.words('russian')

noun = ['S-PRO', 'NUM=n', 'S', 'NONLEX']
verb = ['V']
adj = ['ANUM=pl', 'ANUM=n', 'ANUM=m', 'NUM=comp', 'A=comp2', 'A=m', 'A-PRO=f', 'A-PRO=pl', 'A-PRO', 'A=comp', 'A=sg', 'ADV=comp', 'A', 'A=pl', 'A=f', 'PRAEDIC=comp', 'A=n', 'A-PRO=n', 'ANUM=f', 'ADV', 'NUM=f', 'NUM=m']

#Из новости делаем токены
tokens_texts_news = []

for text in tqdm(text_list):
    text_1 = re.sub('[^\w\s]', '', text).lower()
    text_2 = re.sub('\n', ' ', text_1)
    if text_2.strip() != '':
        text_3 = word_tokenize(text_2.strip())
        text_4 = pos_tag(text_3, lang = 'rus')
        text_5 = []
        for text_teg in text_4:
            if text_teg[1] in noun:
                text_5.append(WordNetLemmatizer().lemmatize(text_teg[0], pos = 'n'))
            elif text_teg[1] in verb:
                text_5.append(WordNetLemmatizer().lemmatize(text_teg[0], pos = 'v'))
            elif text_teg[1] in adj:
                text_5.append(WordNetLemmatizer().lemmatize(text_teg[0], pos = 'a'))
        if len(text_5) > 0:
            text_6 = []
            for word in text_5:
                if word not in stop_words:
                    text_6.append(word)
    if len(text_6) >= 5:
        tokens_texts_news.append(text_6)

  0%|          | 0/188 [00:00<?, ?it/s]

  0%|          | 0/18769 [00:00<?, ?it/s]

In [ ]:
#Создаем Датафрейм
df_main_with_date = pd.DataFrame(list(zip(tokens_texts_news,date_list)), columns = ['Токены, описывающие новость', 'Дата'])
# Добавляем читаемую Дату
clear_date = []
for n in df_main_with_date ['Дата']:
    time = datetime.utcfromtimestamp(n).strftime('%Y-%m-%d')
    clear_date.append(time)
df_main_with_date['Читаемая дата'] = clear_date
df_main_with_date
#Берем данные токенов не по конкретной новости, а по дню
slovar = {}
for i,element in df_main_with_date.iterrows():
    print(i)
    if not str(element['Читаемая дата']) in slovar.keys():
        slovar[str(element['Читаемая дата'])]= []
        for j,n in df_main_with_date.iterrows():
            if n['Читаемая дата'] == element['Читаемая дата']:
                slovar[str(element['Читаемая дата'])] += (n['Токены, описывающие новость'])
# Получаем новый датафрей
df_data = pd.DataFrame(slovar.items(), columns=['Дата', 'Токены'])

In [ ]:
# Собираем токены по крипте
params = {'access_token':token,
          'v':'5.130',
          'domain':domain_special,
          'count':1
         }
inf_count = get('https://api.vk.com/method/wall.get', params = params).json()['response']['count']
offset_list = list(range(0,inf_count+1,100))
params['count'] = 100
text_list = []
for off in tqdm(offset_list):
    params['offset'] = off
    inf = get('https://api.vk.com/method/wall.get', params = params).json()['response']['items']
    for post in inf:
        if 'text' in list(post.keys()):
            text_list.append(post['text'])
            
stop_words = stopwords.words('russian')

noun = ['S-PRO', 'NUM=n', 'S', 'NONLEX']
verb = ['V']
adj = ['ANUM=pl', 'ANUM=n', 'ANUM=m', 'NUM=comp', 'A=comp2', 'A=m', 'A-PRO=f', 'A-PRO=pl', 'A-PRO', 'A=comp', 'A=sg', 'ADV=comp', 'A', 'A=pl', 'A=f', 'PRAEDIC=comp', 'A=n', 'A-PRO=n', 'ANUM=f', 'ADV', 'NUM=f', 'NUM=m']

tokens_texts = []

for text in tqdm(text_list):
    text_1 = re.sub('[^\w\s]', '', text).lower()
    text_2 = re.sub('\n', ' ', text_1)
    if text_2.strip() != '':
        text_3 = word_tokenize(text_2.strip())
        text_4 = pos_tag(text_3, lang = 'rus')
        text_5 = []
        for text_teg in text_4:
            if text_teg[1] in noun:
                text_5.append(WordNetLemmatizer().lemmatize(text_teg[0], pos = 'n'))
            elif text_teg[1] in verb:
                text_5.append(WordNetLemmatizer().lemmatize(text_teg[0], pos = 'v'))
            elif text_teg[1] in adj:
                text_5.append(WordNetLemmatizer().lemmatize(text_teg[0], pos = 'a'))
        if len(text_5) > 0:
            text_6 = []
            for word in text_5:
                if word not in stop_words:
                    text_6.append(word)
    if len(text_6) >= 5:
        tokens_texts.extend(text_6)
        
tematik_tokenns_count = Counter(tokens_texts)
tematik_tokenns_count_2 = {}
for k,v in tematik_tokenns_count.items():
    if v > int(sum(list(tematik_tokenns_count.values())) / len(list(tematik_tokenns_count.values()))):
        tematik_tokenns_count_2[k] = v

df_special = pd.DataFrame(tematik_tokenns_count_2.items())
df_special.to_excel('kripta.xlsx')

In [8]:
df_tematik = pd.read_excel('kripta.xlsx')
df_data = pd.read_excel('to_date.xlsx')

tokens_texts_news = df_data['Токены']

k_1_list = []
k_2_list = []
k_3_list = []
k_4_list = []
k_5_list = []

for post in tqdm(tokens_texts_news):
    crossing_tokkens_list = list(set(post) & set(df_tematik[0]))
# Квантификатор 1 - длина списка общих токенов
    k_1_list.append(len(crossing_tokkens_list))
# Квантификатор 2 - доля общих токенов в массиве токенов, описывающих поисковой объект
    k_2_list.append(len(crossing_tokkens_list) / len(post))
# Квантификатор 3 - доля общих токенов в массиве токенов, описывающих тематическую компоненты
    k_3_list.append(len(crossing_tokkens_list) / len(df_tematik[0]))
# Квантификатор 4 - число общих токенов, скорректирвоанное на удельный вес
    crossing_tokens_imp_list = []
    for token in crossing_tokkens_list:
        crossing_tokens_imp_list.append(df_tematik[df_tematik[0] == token].iloc[0][1])
    k_4_list.append(sum(crossing_tokens_imp_list))
# Квантификатор 5 - доля общих токенов в массиве токенов, описывающих тематическую компоненты, скорректированная на удельный вес
    crossing_tokens_imp_list = []
    for token in crossing_tokkens_list:
        crossing_tokens_imp_list.append(df_tematik[df_tematik[0] == token].iloc[0][1])
    k_5_list.append(sum(crossing_tokens_imp_list) / sum(list(df_tematik[1])))
    
df_final = pd.DataFrame(list(zip(tokens_texts_news,k_1_list,k_2_list,k_3_list,k_4_list,k_5_list)), columns = ['Токены, описывающие новость', 'Длина списка общих токенов', 'Доля общих токенов в массиве токенов, описывающих поисковой объект', 'Доля общих токенов в массиве токенов, описывающих тематическую компоненты', 'Число общих токенов, скорректирвоанное на удельный вес', 'Доля общих токенов в массиве токенов, описывающих тематическую компоненты, скорректированная на удельный вес'])
df_final

  0%|          | 0/1902 [00:00<?, ?it/s]

,"Токены, описывающие новость",Длина списка общих токенов,"Доля общих токенов в массиве токенов, описывающих поисковой объект","Доля общих токенов в массиве токенов, описывающих тематическую компоненты","Число общих токенов, скорректирвоанное на удельный вес","Доля общих токенов в массиве токенов, описывающих тематическую компоненты, скорректированная на удельный вес"
0,"['используете', 'yandexzen', 'подписывайтесь',...",18,0.004694,0.001644,740,0.001026
1,"['райффайзенбанк', 'исключил', 'покупку', 'роз...",6,0.009569,0.000548,177,0.000245
2,"['похоже', 'биткоин', 'столкнуться', 'распрода...",16,0.004731,0.001461,672,0.000932
3,"['сегодня', 'индекс', 'мосбиржи', 'упал', 'инд...",18,0.003662,0.001644,716,0.000993
4,"['движение', 'продолжается', 'траектория', 'по...",19,0.004422,0.001735,758,0.001051
...,...,...,...,...,...,...
1897,"['думаете', 'решении', 'премьерминистра', 'гре...",5,0.033113,0.000457,164,0.000227
1898,"['речь', 'идет', 'деньгах', 'становятся', 'вер...",6,0.021429,0.000548,177,0.000245
1899,"['думаете', 'оправдаются', 'ожидания', 'решени...",6,0.013187,0.000548,177,0.000245
1900,"['команда', 'forexpros', 'опечалена', 'потерей...",13,0.037464,0.001187,521,0.000723


In [9]:
df_final ['Дата'] = df_data ['Дата']
df_final

,"Токены, описывающие новость",Длина списка общих токенов,"Доля общих токенов в массиве токенов, описывающих поисковой объект","Доля общих токенов в массиве токенов, описывающих тематическую компоненты","Число общих токенов, скорректирвоанное на удельный вес","Доля общих токенов в массиве токенов, описывающих тематическую компоненты, скорректированная на удельный вес",Дата
0,"['используете', 'yandexzen', 'подписывайтесь',...",18,0.004694,0.001644,740,0.001026,2019-10-22
1,"['райффайзенбанк', 'исключил', 'покупку', 'роз...",6,0.009569,0.000548,177,0.000245,2021-11-24
2,"['похоже', 'биткоин', 'столкнуться', 'распрода...",16,0.004731,0.001461,672,0.000932,2021-11-23
3,"['сегодня', 'индекс', 'мосбиржи', 'упал', 'инд...",18,0.003662,0.001644,716,0.000993,2021-11-22
4,"['движение', 'продолжается', 'траектория', 'по...",19,0.004422,0.001735,758,0.001051,2021-11-19
...,...,...,...,...,...,...,...
1897,"['думаете', 'решении', 'премьерминистра', 'гре...",5,0.033113,0.000457,164,0.000227,2014-04-11
1898,"['речь', 'идет', 'деньгах', 'становятся', 'вер...",6,0.021429,0.000548,177,0.000245,2014-04-10
1899,"['думаете', 'оправдаются', 'ожидания', 'решени...",6,0.013187,0.000548,177,0.000245,2014-04-09
1900,"['команда', 'forexpros', 'опечалена', 'потерей...",13,0.037464,0.001187,521,0.000723,2014-04-08


In [12]:
# Собираем данные по котировкам биткоина
data = yf.download("BTC-USD", start="2014-04-07", end="2021-11-24")
BTC = pd.read_excel('BTC.xlsx')
BTC

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100
...,...,...,...,...,...,...,...
2616,2021-11-19,56896.128906,58351.113281,55705.179688,58119.578125,58119.578125,38702407772
2617,2021-11-20,58115.082031,59859.878906,57469.726562,59697.195312,59697.195312,30624264863
2618,2021-11-21,59730.507812,60004.425781,58618.929688,58730.476562,58730.476562,26123447605
2619,2021-11-22,58706.847656,59266.359375,55679.839844,56289.289062,56289.289062,35036121783


In [19]:
#Добавляем читаемую дату
clear_date = []
for n in BTC ['Date']:
    n = datetime.timestamp(n)
    time = datetime.utcfromtimestamp(n).strftime('%Y-%m-%d')
    clear_date.append(time)
BTC['Дата'] = clear_date
BTC.to_excel('BTC_with_date.xlsx')

In [22]:
#Создаемой единый датафрейм
res = df_final.merge(BTC, on=['Дата'])
res.to_excel('res_2.xlsx')

In [25]:
#Смотрим на корреляцию
x_simple = np.array(res['Доля общих токенов в массиве токенов, описывающих поисковой объект'])
y_simple = np.array(res['Close'])
my_rho = np.corrcoef(x_simple, y_simple)

In [66]:
#Матрица корреляций
corr = res.corr()
corr.style.background_gradient(cmap='coolwarm')

,Длина списка общих токенов,"Доля общих токенов в массиве токенов, описывающих поисковой объект","Доля общих токенов в массиве токенов, описывающих тематическую компоненты","Число общих токенов, скорректирвоанное на удельный вес","Доля общих токенов в массиве токенов, описывающих тематическую компоненты, скорректированная на удельный вес",Close,Adj Close,Volume
Длина списка общих токенов,1.000000,-0.569350,1.000000,0.960609,0.960609,0.306180,0.306180,0.331981
"Доля общих токенов в массиве токенов, описывающих поисковой объект",-0.569350,1.000000,-0.569350,-0.513453,-0.513453,-0.420850,-0.420850,-0.471438
"Доля общих токенов в массиве токенов, описывающих тематическую компоненты",1.000000,-0.569350,1.000000,0.960609,0.960609,0.306180,0.306180,0.331981
"Число общих токенов, скорректирвоанное на удельный вес",0.960609,-0.513453,0.960609,1.000000,1.000000,0.267670,0.267670,0.291613
"Доля общих токенов в массиве токенов, описывающих тематическую компоненты, скорректированная на удельный вес",0.960609,-0.513453,0.960609,1.000000,1.000000,0.267670,0.267670,0.291613
Close,0.306180,-0.420850,0.306180,0.267670,0.267670,1.000000,1.000000,0.745368
Adj Close,0.306180,-0.420850,0.306180,0.267670,0.267670,1.000000,1.000000,0.745368
Volume,0.331981,-0.471438,0.331981,0.291613,0.291613,0.745368,0.745368,1.000000


In [107]:
#Метод ближайших соседей
X, y = res['Доля общих токенов в массиве токенов, описывающих тематическую компоненты, скорректированная на удельный вес'], res['Close']

knn = KNeighborsRegressor(n_neighbors = 5, weights = 'uniform', p = 2)
# Создаем тренировочную и тестовую выборку, 30% на тест
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

#Нормируем
scaler = StandardScaler()
X_train = scaler.fit_transform(pd.DataFrame(X_train))
y_train = scaler.fit_transform(pd.DataFrame(y_train))
X_test = scaler.fit_transform(pd.DataFrame(X_test))
y_test = scaler.fit_transform(pd.DataFrame(y_test))
#Подбор параметров уменьшил среднеквадратичную ошибку
mean_squared_error(y_test, grid_searcher_knn.predict(X_test))
# Тренируем и предсказываем
knn.fit(X_train, y_train)
print(mean_squared_error(y_test, knn.predict(X_test)))

# Подбираем оптимальные параметры
grid_searcher_knn = GridSearchCV(
    KNeighborsRegressor(),
    param_grid = {
        'n_neighbors': range(1,20),
        'weights': ['uniform', 'distance'],
        'p': range(1,5)
    },
    cv = 5
)

grid_searcher_knn.fit(X_train, y_train)
grid_searcher_knn.best_params_

0.8649677203342759


{'n_neighbors': 17, 'p': 1, 'weights': 'uniform'}

In [108]:
#Подбор параметров уменьшил среднеквадратичную ошибку
mean_squared_error(y_test, grid_searcher_knn.predict(X_test))

0.7806451590640612

In [110]:
# Линейная регрессия
lr = LinearRegression()
lr.fit(X_train, y_train)
y_train_pred_lr = lr.predict(X_train)
y_pred_lr = lr.predict(X_test)
mean_squared_error(y_test, lr.predict(X_test))

0.9273165755236454

In [111]:
#Метод ближайших соседей
X, y = res['Доля общих токенов в массиве токенов, описывающих поисковой объект'], res['Close']

knn = KNeighborsRegressor(n_neighbors = 5, weights = 'uniform', p = 2)
# Создаем тренировочную и тестовую выборку, 30% на тест
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

#Нормируем
scaler = StandardScaler()
X_train = scaler.fit_transform(pd.DataFrame(X_train))
y_train = scaler.fit_transform(pd.DataFrame(y_train))
X_test = scaler.fit_transform(pd.DataFrame(X_test))
y_test = scaler.fit_transform(pd.DataFrame(y_test))

# Тренируем и предсказываем
knn.fit(X_train, y_train)
print(mean_squared_error(y_test, knn.predict(X_test)))

# Подбираем оптимальные параметры
grid_searcher_knn = GridSearchCV(
    KNeighborsRegressor(),
    param_grid = {
        'n_neighbors': range(1,20),
        'weights': ['uniform', 'distance'],
        'p': range(1,5)
    },
    cv = 5
)

grid_searcher_knn.fit(X_train, y_train)
grid_searcher_knn.best_params_

0.7693659193170171


{'n_neighbors': 18, 'p': 1, 'weights': 'uniform'}

In [112]:
#Подбор параметров уменьшил среднеквадратичную ошибку
mean_squared_error(y_test, grid_searcher_knn.predict(X_test))

0.6818644276820457

In [113]:
# Линейная регрессия
lr = LinearRegression()
lr.fit(X_train, y_train)
y_train_pred_lr = lr.predict(X_train)
y_pred_lr = lr.predict(X_test)
mean_squared_error(y_test, lr.predict(X_test))

0.8265559622026516

In [116]:
#Деревья
def rmse(y_true, y_pred):
    error = (y_true - y_pred) ** 2
    return np.sqrt(np.mean(error))

cbm = CatBoostRegressor(iterations=100, max_depth=4, learning_rate=0.01)
cbm.fit(X_train, y_train)

y_pred_cbm = cbm.predict(X_test)
y_train_pred_cbm = cbm.predict(X_train)
y_pred_cbm = cbm.predict(X_test)

print('Train RMSE GB = %.4f' %rmse(y_train, y_train_pred_cbm))
print('Test RMSE GB = %.4f' %rmse(y_test, y_pred_cbm))

0:	learn: 0.9963892	total: 628us	remaining: 62.2ms
1:	learn: 0.9928769	total: 1.37ms	remaining: 67ms
2:	learn: 0.9893682	total: 2.6ms	remaining: 84.2ms
3:	learn: 0.9860927	total: 3.63ms	remaining: 87.3ms
4:	learn: 0.9827092	total: 4.5ms	remaining: 85.6ms
5:	learn: 0.9794550	total: 5.28ms	remaining: 82.8ms
6:	learn: 0.9762808	total: 6.02ms	remaining: 80ms
7:	learn: 0.9730521	total: 6.76ms	remaining: 77.7ms
8:	learn: 0.9697918	total: 7.54ms	remaining: 76.3ms
9:	learn: 0.9668456	total: 8.31ms	remaining: 74.8ms
10:	learn: 0.9637176	total: 9.07ms	remaining: 73.4ms
11:	learn: 0.9606981	total: 9.83ms	remaining: 72.1ms
12:	learn: 0.9576990	total: 10.5ms	remaining: 70.2ms
13:	learn: 0.9547630	total: 11.2ms	remaining: 68.9ms
14:	learn: 0.9520136	total: 11.9ms	remaining: 67.5ms
15:	learn: 0.9493449	total: 12.6ms	remaining: 66.2ms
16:	learn: 0.9465592	total: 13.3ms	remaining: 64.7ms
17:	learn: 0.9439429	total: 13.8ms	remaining: 62.9ms
18:	learn: 0.9411998	total: 14.4ms	remaining: 61.4ms
19:	learn:

In [117]:
def select_weight(y_true, y_pred_1, y_pred_2):
    metric = []
    grid = np.linspace(0,1,1000)
    for w_0 in grid:
        w_1 = 1 - w_0
        y = y_pred_1 * w_0 + y_pred_2 * w_1
        
        metric.append([rmse(y_true, y), w_0, w_1])
    
    
    
    return metric
rmse_blending_train, w_0, w_1 = min (select_weight(y_train, y_train_pred_cbm, y_train_pred_lr), key = lambda x: x[0])
rmse_blending_train
print("Test RMSE blending - %.4f"%rmse(y_test, y_pred_cbm*w_0 + y_pred_lr*w_1))

Test RMSE blending - 0.9092
